RFM是典型的用户分层方法，是评估用户消费能力、衡量用户贡献价值的重要工具。RFM代表的是最近一次消费时间间隔（Recency）、消费频率（Frequency）和消费金额（Monetary）。本案例将利用Pandas建立用户消费RFM模型，实现精细化运营。

# 构造数据

In [1]:
import pandas as pd

import faker 

In [2]:
f = faker.Faker('zh-cn')

In [5]:
df = pd.DataFrame({
    '用户':[f.name() for i in range(20000)],
    '购买日期':[f.date_between(start_date='-1y',end_date='today') for i in range(20000)],
    '金额':[f.random_int(10, 100) for i in range(20000)]
})

In [6]:
df.head()

,用户,购买日期,金额
0,王瑞,2021-06-21,65
1,李辉,2021-09-24,12
2,徐彬,2022-01-08,40
3,梁瑞,2021-10-26,41
4,林文,2021-07-15,95


In [7]:
# 查看数据类型
df.dtypes

用户      object
购买日期    object
金额       int64
dtype: object

In [8]:
# 数据类型转换
df = df.astype({'购买日期':'datetime64[ns]'})
df.dtypes

用户              object
购买日期    datetime64[ns]
金额               int64
dtype: object

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   用户      20000 non-null  object        
 1   购买日期    20000 non-null  datetime64[ns]
 2   金额      20000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 468.9+ KB


## 首先来计算R值。

R为最后一次购买时间距今的天数，R值越大代表用户越有可能处于沉睡状态，流失风险越大：

先对用户分组，分组后取每组用户最近购买时间（时间的最大值），然后用今日减去最近购买时间，就得到了最近购买间隔天数。

In [9]:
# r为购买间隔天数
r = (
    df.groupby('用户')
    .apply(lambda x:(pd.Timestamp('today')-x['购买日期'].max()))
    .dt
    .days
)
r

用户
丁东      51
丁丽     175
丁丽丽    146
丁丽华    223
丁丽娟    155
      ... 
龚阳     232
龚雪梅    133
龚雷     181
龚静     227
龚鹏     311
Length: 9340, dtype: int64

In [10]:
type(r)

pandas.core.series.Series

## 接下来计算F值。

F值是消费频率，消费频次越高代表用户黏性越强。我们将同一天购买多次的情况算作一次。算法也是先对用户分组，然后取购买日期的不重复数量：

In [11]:
# f为购买次数，一天多次算一次
f = (
    df.groupby(['用户'])
#     .apply(lambda x:x['购买日期'])
    .apply(lambda x:x['购买日期'].nunique())
)
f.sort_values()

用户
沈浩      1
童建国     1
童帆      1
童刚      1
童佳      1
       ..
李雪梅    19
王亮     19
王华     21
张淑英    21
李欢     21
Length: 9340, dtype: int64

## 计算M值

我们这里计算用户每次购物的平均金额，即用户总金额/用户购买次数。由于前面已经算出购买次数，因此我们在合并数据时再计算M值，这里先计算出每个用户的总金额

In [12]:
# m为平均每次的购买金额
df.groupby(['用户']).sum()['金额']

用户
丁东     75
丁丽     59
丁丽丽    35
丁丽华    61
丁丽娟    72
       ..
龚阳     47
龚雪梅    85
龚雷     29
龚静     43
龚鹏     15
Name: 金额, Length: 9340, dtype: int64

接下来将RFM数据合并。由于我们之前在计算R值和F值后都是以用户名称为索引的，因此直接用两个Series构造DataFrame，同时算出M值：

## 合并RFM

In [13]:
len(r)

9340

In [14]:
len(f)

9340

In [15]:
(
    pd.DataFrame({'r':r,'f':f})
    # m 为总金额/购买次数
    .assign(m=lambda x:df.groupby(['用户']).sum()['金额']/x.f)
)

,r,f,m
用户,,,
丁东,51,2,37.5
丁丽,175,1,59.0
丁丽丽,146,1,35.0
丁丽华,223,2,30.5
丁丽娟,155,1,72.0
...,...,...,...
龚阳,232,1,47.0
龚雪梅,133,1,85.0
龚雷,181,1,29.0


这样，每个用户的RFM值就计算出来了。接着给RFM打分，为了方便演示，采用3分制，将RFM的值分为三个等级。
R值使用pd.qcut()平均分为三段，R越大代表间隔时间越长，对间隔近的打3分，次之打2分，最远的打1分。
F值和M值越大越好，因此我们用pd.cut()人工分段，分别打1、2、3分。代码如下：

In [17]:
(
    pd.DataFrame({'r':r,'f':f})
    # m 为总金额/购买次数
    .assign(m=lambda x:df.groupby(['用户']).sum()['金额']/x.f)
    .assign(r_s=lambda x:pd.qcut(x.r,q=3,labels=[3,2,1])) # 分三段qcut
    .assign(f_s=lambda x:pd.cut(x.f,bins=[0,2,5,float('inf')]),labels=[1,2,3],right=False)  # 正无穷大
    .assign(m_s=lambda x:pd.cut(x.m,bins=[0,30,60,float('inf')]),labels=[1,2,3],right=False)
)

ValueError: Length of values (3) does not match length of index (9340)